In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import gc

In [ ]:
items_per_file = 2 #int(input("Bitte geben sie die Anzahl der Einträge pro Datei an:"))
name ="center_single"  #input("Bitte den Namen der zu erstellenden Datein angeben:")
data_source =r'C:\Users\ludwi\Desktop\JuFo_DeepLearning\Generiere_daten\test' #input("Bitte .npz Zielordner angeben:")
save_to =r'C:\Users\ludwi\Desktop\JuFo_DeepLearning\Generiere_daten\test' #input("Bitte Zielordner für TFRecord angeben:")

In [ ]:
def load_data_py(file_path):
    return np.load(file_path.numpy())['image'], np.load(file_path.numpy())['target']

def load_data(file_path):
    return tf.py_function(func=load_data_py, inp=[file_path], Tout=[tf.int16, tf.int16])

def reshape(x, y):
    X = tf.reshape(x, [16, 160, 160])
    Y = tf.reshape(y, [])
    return X, Y

In [ ]:
dataset = tf.data.Dataset.list_files(data_source + '\*.npz')
dataset = dataset.map(load_data, tf.data.experimental.AUTOTUNE)
dataset = dataset.map(reshape, tf.data.experimental.AUTOTUNE)

In [ ]:
def serialize_example(image, label):
    feature = {
        "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(image).numpy()])),
        "label": tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(label).numpy()])),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def tf_serialize_example(image, label):
    data = tf.py_function(func=serialize_example, inp=[image, label], Tout=tf.string)
    return tf.reshape(data, ())

In [ ]:
dataset = dataset.map(tf_serialize_example, tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(items_per_file)

In [ ]:
def write_generator():
    i = 0
    iterator = iter(dataset)
    optional = iterator.get_next_as_optional()
    while optional.has_value().numpy():
        ds = optional.get_value()
        optional = iterator.get_next_as_optional()
        batch_ds = tf.data.Dataset.from_tensor_slices(ds)
        writer = tf.data.experimental.TFRecordWriter(save_to + "\\" + name + "-" + str(i) + ".tfrecord", compression_type='GZIP')#compression_type='GZIP'
        i += 1
        yield batch_ds, writer, i, optional.has_value().numpy()
    return

In [ ]:
for data, wri, i in write_generator():
    start_time = time.time()
    wri.write(data)
    print("Dauer war: ", time.time() - start_time, "s", "\t", name + "-" + str(i) + ".tfrecord")
    gc.collect()

In [ ]:
print("______________________________________________________")
print("*fertig*")